In [ ]:
from __future__ import division
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import numpy as np,  matplotlib.pyplot as plt, gc, math
from time import time
from pylab import plot, show, legend
from PF import *
from OU import *
plt.rcParams['figure.figsize'] = (15.0, 3.0)

In [ ]:
y = np.load("data/y_OU.npy")
x_0 = np.load("data/x_0_OU.npy")
theta = np.load("data/theta_OU.npy")
data = dict(x_0=x_0, y=y)
def test_fn(x) : return x
propagate, potential = propagate_OU, potential_OU

In [ ]:
# pow_list = np.arange(4,7)
d_list = 100*np.arange(1,6)
# N_list = 2**pow_list
N_list = np.arange(1,11)*1000
rep = 10
logNC_bpf, testfn_bpf = np.zeros((rep,len(N_list))), np.zeros((rep,len(N_list)))
# logNC_airpf, testfn_airpf = np.zeros((len(N_list),rep)), np.zeros((len(N_list),rep))
logNC_alpha, testfn_alpha = np.zeros((rep,len(N_list),len(d_list))), np.zeros((rep,len(N_list),len(d_list)))
for r in trange(rep) :
    for (i,N) in enumerate(N_list) :
        #Bootstrap PF:
        logNC, testfn, _ = bootstrap_PF(data, theta, potential, propagate, test_fn, N, store_paths=False)
        logNC_bpf[r,i], testfn_bpf[r,i] = logNC[-1], testfn[-1]
#         #AIRPF:
#         A = A_(int(math.log(N,2)))
#         logNC, testfn, _ = AIRPF(data, theta, potential, propagate, test_fn, A, store_paths=False)
#         logNC_airpf[i,r], testfn_airpf[i,r] = logNC[-1], testfn[-1]
        #alpha SMC:
        for (j,d) in enumerate(d_list) :
            logNC, testfn = alphaSMC(data, theta, potential, propagate, test_fn, N, d, "random")
            logNC_alpha[r,i,j], testfn_alpha[r,i,j] = logNC[-1], testfn[-1]
        del logNC, testfn, _
        gc.collect()

In [ ]:
i = 9
np.save("data/logNC_bpf"+str(i), logNC_bpf)
np.save("data/logNC_alpha"+str(i), logNC_alpha)
# np.save("data/logNC_airpf"+str(i), logNC_airpf)

np.save("data/testfn_bpf"+str(i), testfn_bpf)
np.save("data/testfn_alpha"+str(i), testfn_alpha)
# np.save("data/testfn_airpf"+str(i), testfn_airpf)